In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModel

/home/suman12/300-days-challenge/300-days-challenge/300cha/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("/home/suman12/300-days-challenge/data.json","r") as f:
  data=json.load(f)

In [12]:
data

[{'id': 'K1', 'key': 'JSON{Doctor’s Name: ?}', 'value': 'Dr Anup Joshi'},
 {'id': 'K2',
  'key': 'JSON{Appointment Time: ?}',
  'value': 'Tuesday 15 July 2025, 01:00 PM'},
 {'id': 'K3',
  'key': 'JSON{Doctor’s Location: ?}',
  'value': 'Om Hospital, Chabahil'},
 {'id': 'K10', 'key': 'JSON{Doctor’s Name: ?}', 'value': 'Dr Anjali Mehta'},
 {'id': 'K5',
  'key': 'JSON{Appointment Time: ?}',
  'value': 'Friday 04 July 2025, 12:00 PM'},
 {'id': 'K6',
  'key': 'JSON{Doctor’s Location: ?}',
  'value': 'Grande Hospital, Tokha'},
 {'id': 'K19', 'key': 'JSON{Doctor’s Name: ?}', 'value': 'Dr Milan Gurung'},
 {'id': 'K8',
  'key': 'JSON{Appointment Time: ?}',
  'value': 'Thursday 03 July 2025, 12:00 PM'},
 {'id': 'K9',
  'key': 'JSON{Doctor’s Location: ?}',
  'value': 'Grande Hospital, Tokha'},
 {'id': 'K28', 'key': 'JSON{Doctor’s Name: ?}', 'value': 'Dr Rina Shrestha'},
 {'id': 'K11',
  'key': 'JSON{Appointment Time: ?}',
  'value': 'Thursday 03 July 2025, 09:00 AM'},
 {'id': 'K12',
  'key': 'JSO

In [22]:
print(type(data))



<class 'list'>


In [23]:
import pandas as pd
df = pd.DataFrame(data)
print(df.head())


    id                         key                           value
0   K1      JSON{Doctor’s Name: ?}                   Dr Anup Joshi
1   K2   JSON{Appointment Time: ?}  Tuesday 15 July 2025, 01:00 PM
2   K3  JSON{Doctor’s Location: ?}           Om Hospital, Chabahil
3  K10      JSON{Doctor’s Name: ?}                 Dr Anjali Mehta
4   K5   JSON{Appointment Time: ?}   Friday 04 July 2025, 12:00 PM


In [26]:
new_data=df.key
new_data

0          JSON{Doctor’s Name: ?}
1       JSON{Appointment Time: ?}
2      JSON{Doctor’s Location: ?}
3          JSON{Doctor’s Name: ?}
4       JSON{Appointment Time: ?}
                  ...            
145     JSON{Appointment Time: ?}
146    JSON{Doctor’s Location: ?}
147        JSON{Doctor’s Name: ?}
148     JSON{Appointment Time: ?}
149    JSON{Doctor’s Location: ?}
Name: key, Length: 150, dtype: object

In [27]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [28]:
model_name = "Qwen/Qwen3-Embedding-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
model = model.to(device)

In [29]:
def get_embedding(text: str) -> torch.Tensor:
    """Generate embedding for a single text input"""
    # Tokenize with model's requirements
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    ).to(device)
    
    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract last hidden states
    last_hidden_state = outputs.last_hidden_state
    
    # Mean pooling to create sentence embedding
    embedding = last_hidden_state.mean(dim=1).squeeze()
    
    return embedding.cpu()  # Return to CPU memory

In [32]:
key_embeddings = {}
for key in new_data:
    embedding = get_embedding(key)
    key_embeddings[key] = embedding.numpy()  # Convert to NumPy array
    print(f"Embedded: '{key}' => Vector shape: {embedding.shape}")

Embedded: 'JSON{Doctor’s Name: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Appointment Time: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Doctor’s Location: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Doctor’s Name: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Appointment Time: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Doctor’s Location: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Doctor’s Name: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Appointment Time: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Doctor’s Location: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Doctor’s Name: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Appointment Time: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Doctor’s Location: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Doctor’s Name: ?}' => Vector shape: torch.Size([1024])
Embedded: 'JSON{Appointment Time: ?}' => Vector s

In [33]:
# Check results
for key, emb in key_embeddings.items():
    print(f"{key}: {emb.shape}") 

# Example output:
# name: (1024,)
# age: (1024,)
# city: (1024,)
# pets: (1024,)

JSON{Doctor’s Name: ?}: (1024,)
JSON{Appointment Time: ?}: (1024,)
JSON{Doctor’s Location: ?}: (1024,)


In [34]:
key_embeddings

{'JSON{Doctor’s Name: ?}': array([ 2.7954187 , -2.1706378 , -1.0065917 , ..., -2.3348398 ,
         1.00306   ,  0.24185224], shape=(1024,), dtype=float32),
 'JSON{Appointment Time: ?}': array([-0.71639794,  0.89461267, -0.8938823 , ...,  1.5467436 ,
         1.656104  ,  0.7327456 ], shape=(1024,), dtype=float32),
 'JSON{Doctor’s Location: ?}': array([ 1.6740471 , -0.98419446, -1.006454  , ..., -0.40001956,
         1.4949404 ,  0.36329815], shape=(1024,), dtype=float32)}

In [35]:
print(key_embeddings)

{'JSON{Doctor’s Name: ?}': array([ 2.7954187 , -2.1706378 , -1.0065917 , ..., -2.3348398 ,
        1.00306   ,  0.24185224], shape=(1024,), dtype=float32), 'JSON{Appointment Time: ?}': array([-0.71639794,  0.89461267, -0.8938823 , ...,  1.5467436 ,
        1.656104  ,  0.7327456 ], shape=(1024,), dtype=float32), 'JSON{Doctor’s Location: ?}': array([ 1.6740471 , -0.98419446, -1.006454  , ..., -0.40001956,
        1.4949404 ,  0.36329815], shape=(1024,), dtype=float32)}


In [46]:
vectors_only = list(key_embeddings.values())
print(vectors_only)

[array([ 2.79541874, -2.17063785, -1.00659168, ..., -2.33483982,
        1.00305998,  0.24185224], shape=(1024,)), array([-0.71639794,  0.89461267, -0.89388227, ...,  1.54674363,
        1.65610397,  0.73274559], shape=(1024,)), array([ 1.67404711, -0.98419446, -1.00645399, ..., -0.40001956,
        1.4949404 ,  0.36329815], shape=(1024,))]


In [48]:
# Add embeddings to the data
for entry in data:
    key = entry["key"]
    embedding = key_embeddings.get(key)
    if embedding is not None:
        entry["embedding"] = embedding.tolist()  # convert NumPy array to list

In [49]:
# Save to new JSON file (optional)
with open("enriched_data.json", "w") as f:
    json.dump(data, f, indent=2)

In [50]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-Embedding-0.6B")
model = AutoModel.from_pretrained("Qwen/Qwen3-Embedding-0.6B")

def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    # Usually embeddings come from the last hidden state pooled or CLS token, depends on model
    embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token embedding
    return embeddings.detach().numpy()[0]

In [51]:
# Embed your query
query = "Who is the doctor"
query_vector = embed_text(query)
print(query_vector.shape)  # e.g., (1024,)

(1024,)


In [52]:
query_vector

array([  0.39899698, -15.500072  ,   0.25314695, ...,  -6.286629  ,
        -9.756202  ,  -0.03672666], shape=(1024,), dtype=float32)

In [ ]:
#to check top k similarity

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = {}

# embedding_table is dict: key -> vector
for key, vec in vectors_only.items():
    sim = cosine_similarity([query_vector], [vec])[0][0]
    similarities[key] = sim

# Sort top 10
top_matches = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:10]

for k, score in top_matches:
    print(f"{k}: {score:.4f}")


AttributeError: 'list' object has no attribute 'items'

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# embedding_table is your dict: key -> vector (np.array)
similarities = {}

for key, vec in vectors_only.items():  # use embedding_table, not vectors_only
    sim = cosine_similarity([query_vector], [vec])[0][0]
    similarities[key] = sim



AttributeError: 'list' object has no attribute 'items'